In [80]:
# kissa zahra   i21-0572   AI project

from collections import namedtuple
import random

# defining the course allocation list
sections = ['Section A', 'Section B', 'Section C']
subjects = ['Operating Systems (CS)', 'Database Lab', 'Artificial Intelligence (CS)']
teachers = ['Dr. Ahmad Raza Shahid', 'Prof. Usama Imtiaz', 'Prof. Muhammad Owais Idrees']

course_allocation = []
# every section will be studing 3 courses from the required professor so every professor will max can teach 1 course
for subject, teacher in zip(subjects, teachers):  # to combine two or more iterators
  for section in sections:
    course_allocation.append({'Course': subject, 'Instructor': teacher, 'Section': section})


print(course_allocation)



[{'Course': 'Operating Systems (CS)', 'Instructor': 'Dr. Ahmad Raza Shahid', 'Section': 'Section A'}, {'Course': 'Operating Systems (CS)', 'Instructor': 'Dr. Ahmad Raza Shahid', 'Section': 'Section B'}, {'Course': 'Operating Systems (CS)', 'Instructor': 'Dr. Ahmad Raza Shahid', 'Section': 'Section C'}, {'Course': 'Database Lab', 'Instructor': 'Prof. Usama Imtiaz', 'Section': 'Section A'}, {'Course': 'Database Lab', 'Instructor': 'Prof. Usama Imtiaz', 'Section': 'Section B'}, {'Course': 'Database Lab', 'Instructor': 'Prof. Usama Imtiaz', 'Section': 'Section C'}, {'Course': 'Artificial Intelligence (CS)', 'Instructor': 'Prof. Muhammad Owais Idrees', 'Section': 'Section A'}, {'Course': 'Artificial Intelligence (CS)', 'Instructor': 'Prof. Muhammad Owais Idrees', 'Section': 'Section B'}, {'Course': 'Artificial Intelligence (CS)', 'Instructor': 'Prof. Muhammad Owais Idrees', 'Section': 'Section C'}]


In [81]:
# tuples for the timetable entries
TimetableEntry = namedtuple("TimetableEntry", ["course", "professor", "section", "is_lab", "lecture_day", "lecture_timeslot", "room", "room_size"])

# time constants
lab_session_timing = "14:30-17:30"
theory_class_timings = ["08:30-09:50", "10:05-11:25", "11:40-13:00", "13:15-14:35"]
days_in_week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]

# room allocations from C304 to C313
additional_rooms = [f"C {room}" for room in range(304, 314)]

In [82]:
# Generate 1 chromosome
def generate_timetable(course_allocation):
  timetable = []
  courses_assigned = {section: {'Lab': 0, 'Theory': 0, 'last_theory_day': None} for section in set(course['Section'] for course in course_allocation)}
  room_index = 0  # to select the room cyclically

  for course in course_allocation:
    section = course['Section']
    # to see if its a lab or course
    is_lab = "Lab" in course["Course"]

    # assign it a lecture
    if is_lab:
        lecture_day = random.sample(range(len(days_in_week)), 1)  # only one lab session per week
    else:
        lecture_day = random.sample(range(len(days_in_week)), 2)  # will randomly  select 2 days for theory classes
    lecture_day.sort() # sort the days to maintain order of the week days
    max_iterations = 1000
    iterations = 0

    # to make sure not more than 1 theory lecture is assigned
    while (courses_assigned[section]['Theory'] >= 2 and not is_lab) or (courses_assigned[section]['last_theory_day'] and any(day - 1 in lecture_day for day in courses_assigned[section]['last_theory_day'])):
        if is_lab:
            lecture_day = random.sample(range(len(days_in_week)), 1)
        else:
            lecture_day = random.sample(range(len(days_in_week)), 2)
        lecture_day.sort()
        iterations += 1
        if iterations > max_iterations:
            break

    # if its not a lab so count = count + 1
    if not is_lab:
        courses_assigned[section]['Theory'] += 1
        courses_assigned[section]['last_theory_day'] = lecture_day

    if is_lab:
        lecture_timeslot = 0  # fix timeing of all the lab sessions
    else:
        lecture_timeslot = random.randint(0, len(theory_class_timings) - 1)

    # assigning a room and room size
    if is_lab:
        room = additional_rooms[room_index % len(additional_rooms)]
        room_index += 1
        room_size = 60  #  lab rooms have smaller capacity
    else:
        room = f"C {random.randint(201, 303)}"
        # Assign room size based on section strength
        section_strength = random.randint(40, 120)   # selecting the room base on the strength of the class room
        room_size = 60 if section_strength < 60 else 120

    timetable_entry = TimetableEntry(course["Course"], course["Instructor"], course["Section"], is_lab,
                                      lecture_day, lecture_timeslot, room, room_size)
    timetable.append(timetable_entry)   # appending the time table

  return timetable

In [83]:
# generate a time table that follows all the constratins
timetable = generate_timetable(course_allocation)


course_index_map = {course: index for index, course in enumerate(set(course['Course'] for course in course_allocation))}
professor_index_map = {professor: index for index, professor in enumerate(set(course['Instructor'] for course in course_allocation))}
# map the professor and courses to unique indecies

def generate_chromosomes_for_section(section, section_timetable):
  chromosomes = []
  for entry in section_timetable:   # create a chromose
    course_index = entry.course  # with the course name
    is_lab = 1 if entry.is_lab else 0  # check if its a lab or not
    professor_index = entry.professor  # put the professor name
    lecture_day = days_in_week[entry.lecture_day[0]]  # first day of the lecture
    lecture_timeslot = theory_class_timings[entry.lecture_timeslot] if not entry.is_lab else lab_session_timing  # timing of the lecture or theoyr class
    # if its a lab class so from 2:30 to 5:30
    lecture_room_index = entry.room  # the room that is allocated
    lecture_room_size = entry.room_size  # also the room size

    # check for the second slot
    second_lecture_day = days_in_week[entry.lecture_day[1]] if len(entry.lecture_day) > 1 else days_in_week[entry.lecture_day[0]]  # the day jab second class hoti hai
    second_lecture_timeslot = theory_class_timings[entry.lecture_timeslot] if not entry.is_lab else lab_session_timing   # time slot of the second lecture
    # which will be same as the 1st one
    second_lecture_room_index = entry.room  # the room that is allocated
    second_lecture_room_size = entry.room_size    # also the room size

    # our chromose willl have these values
    chromosome = [course_index, is_lab, section, entry.room_size, professor_index, lecture_day, lecture_timeslot, lecture_room_index, lecture_room_size,
                second_lecture_day, second_lecture_timeslot, second_lecture_room_index, second_lecture_room_size]

    chromosomes.append(chromosome)

  return chromosomes



In [84]:
def encode_chromosome(chromosome):
  encoded_chromosome = []

  # converting 1 chromose into binary
  encoded_chromosome.append(bin(course_index_map[chromosome[0]])[2:].zfill(4))  # takes the course index
  encoded_chromosome.append(str(chromosome[1]))  # Labtakes the lab, wheather its a lab or not
  # Encoding Section
  section_char = chromosome[2][8]  # extract the character after 'Section'
  encoded_chromosome.append(bin(ord(section_char))[2:].zfill(8))  # convert to binary and pad to 8 bits
  encoded_chromosome.append(bin(chromosome[3] // 60)[2:].zfill(2))  # divide the rromsize by 60
  encoded_chromosome.append(bin(professor_index_map[chromosome[4]])[2:].zfill(3))  # takes the professor Index

  # Encoding Lecture Day
  lecture_day_binary = ['0'] * 5
  for day in chromosome[5].split(','):
    lecture_day_binary[days_in_week.index(day)] = '1'
  encoded_chromosome.extend(lecture_day_binary)

  # Encoding Lecture Timeslot
  if chromosome[1] == 1:  # For lab timings
    encoded_chromosome.append(bin(len(lab_session_timing))[2:].zfill(5))
  else:# Lecture timing
    encoded_chromosome.append(bin(theory_class_timings.index(chromosome[6]))[2:].zfill(5))

  encoded_chromosome.append(bin(int(chromosome[7].split()[1]))[2:].zfill(3))  # takes lecture Room Index
  encoded_chromosome.append(bin(chromosome[8] // 60)[2:].zfill(2))  # takes lecture Room Size

  # now encoding Second Lecture Day and Timeslot
  if len(chromosome) > 9:
    second_lecture_day_binary = ['0'] * 5
    for day in chromosome[9].split(','):
        second_lecture_day_binary[days_in_week.index(day)] = '1'
    encoded_chromosome.extend(second_lecture_day_binary)

    # Encoding Second Lecture Timeslot
    if chromosome[1] == 1:  # For lab timings
      encoded_chromosome.append(bin(len(lab_session_timing))[2:].zfill(5))
    else: # Lecture timing
      encoded_chromosome.append(bin(theory_class_timings.index(chromosome[10]))[2:].zfill(5))
    encoded_chromosome.append(bin(int(chromosome[11].split()[1]))[2:].zfill(3))  # Second Lecture Room Index
    encoded_chromosome.append(bin(chromosome[12] // 60)[2:].zfill(2))  # Second Lecture Room Size

  return encoded_chromosome



#now decoding them.
def decode_chromosome(encoded_chromosome):
  decoded_chromosome = []

  # decoding the components of the first chromosome
  decoded_chromosome.append(next(key for key, value in course_index_map.items() if value == int(encoded_chromosome[0], 2)))  # getting course Index
  decoded_chromosome.append(int(encoded_chromosome[1]))  # checking if its a lab or not
  # decoding  Section
  section_char = chr(int(encoded_chromosome[2], 2))  # Convert binary to integer and then to character
  decoded_chromosome.append(f'Section {section_char}')  # converting  'Section' to the character
  decoded_chromosome.append(int(encoded_chromosome[3], 2) * 60)  # multiplyoing room size by 60
  decoded_chromosome.append(next(key for key, value in professor_index_map.items() if value == int(encoded_chromosome[4], 2)))  # decoding professor index

  # decoding Lecture Day
  lecture_day = [days_in_week[i] for i, bit in enumerate(encoded_chromosome[5:10]) if bit == '1']
  decoded_chromosome.append(','.join(lecture_day))

  # decoding Lecture Timeslot
  lecture_timeslot_index = int(encoded_chromosome[10], 2)
  if decoded_chromosome[1] == 1:  # For lab sessions
    decoded_chromosome.append(lab_session_timing)  # Lab Timeslot
  else:
    if lecture_timeslot_index < len(theory_class_timings):
      decoded_chromosome.append(theory_class_timings[lecture_timeslot_index])  # Lecture Timeslot
    else:
      decoded_chromosome.append("Invalid timeslot index")

  decoded_chromosome.append(f'C {int(encoded_chromosome[11], 2)}')  # Lecture Room Index
  decoded_chromosome.append(int(encoded_chromosome[12], 2) * 60)  # Lecture Room Size

  # decoding for the  Second Lecture
  if len(encoded_chromosome) > 13:
      # Decoding Second Lecture Day
      second_lecture_day = [days_in_week[i] for i, bit in enumerate(encoded_chromosome[13:18]) if bit == '1']
      decoded_chromosome.append(','.join(second_lecture_day))

      # deconding second Lecture Timeslot
      second_lecture_timeslot_index = int(encoded_chromosome[18], 2)
      if decoded_chromosome[1] == 1:  # For lab timing
          decoded_chromosome.append(lab_session_timing)
      else:
          if second_lecture_timeslot_index < len(theory_class_timings):
              decoded_chromosome.append(theory_class_timings[second_lecture_timeslot_index])
          else:
              decoded_chromosome.append("Invalid timeslot index")

      decoded_chromosome.append(f'C {int(encoded_chromosome[19], 2)}')
      decoded_chromosome.append(int(encoded_chromosome[20], 2) * 60)

  return decoded_chromosome


def printt(course_allocation):
    # Generate timetable entries
    timetable = generate_timetable(course_allocation)

    # generate chromosomes for each section
    for section in set(course['Section'] for course in course_allocation):
        section_timetable = [entry for entry in timetable if entry.section == section]
        chromosomes = generate_chromosomes_for_section(section, section_timetable)
        population = []
        for i in range(5):  # Generate 5 chromosomes for each section so it will be 5 chromose in a population
            population.append(chromosomes)
            print(f"Chromosomes {i+1} for Section {section}:")
            for chromosome in chromosomes:
                encoded_chromosome = encode_chromosome(chromosome)
                decoded_chromosome = decode_chromosome(encoded_chromosome)
                print(f"Original: {chromosome}")
                print(f"Encoded: {encoded_chromosome}")
                print(f"Decoded: {decoded_chromosome}")
                print()


printt(course_allocation)

Chromosomes 1 for Section Section B:
Original: ['Operating Systems (CS)', 0, 'Section B', 120, 'Dr. Ahmad Raza Shahid', 'Monday', '10:05-11:25', 'C 251', 120, 'Thursday', '10:05-11:25', 'C 251', 120]
Encoded: ['0001', '0', '01000010', '10', '000', '1', '0', '0', '0', '0', '00001', '11111011', '10', '0', '0', '0', '1', '0', '00001', '11111011', '10']
Decoded: ['Operating Systems (CS)', 0, 'Section B', 120, 'Dr. Ahmad Raza Shahid', 'Monday', '10:05-11:25', 'C 251', 120, 'Thursday', '10:05-11:25', 'C 251', 120]

Original: ['Database Lab', 1, 'Section B', 60, 'Prof. Usama Imtiaz', 'Monday', '14:30-17:30', 'C 305', 60, 'Monday', '14:30-17:30', 'C 305', 60]
Encoded: ['0000', '1', '01000010', '01', '010', '1', '0', '0', '0', '0', '01011', '100110001', '01', '1', '0', '0', '0', '0', '01011', '100110001', '01']
Decoded: ['Database Lab', 1, 'Section B', 60, 'Prof. Usama Imtiaz', 'Monday', '14:30-17:30', 'C 305', 60, 'Monday', '14:30-17:30', 'C 305', 60]

Original: ['Artificial Intelligence (CS)'

In [86]:
def Fitness(timetable):  # evaluates the fitness of a timetable by checking for clashes in lecture timeslots

  # creates a dictionary to store occupied slots
  occupied_slots = {}

  # iterate over whole time table
  for entry in timetable:
    # check  for clashes in lecture timeslots
    lecture_timeslot = entry.lecture_timeslot
    lecture_day = tuple(entry.lecture_day)
    section = entry.section

    # if its now oocupied initialize it
    if lecture_day not in occupied_slots:
        occupied_slots[lecture_day] = {}

    # if that time slot is already occupied so return true
    if lecture_timeslot in occupied_slots[lecture_day]:
        return True

    # other wise mark the spot as oocupied
    occupied_slots[lecture_day][lecture_timeslot] = section

  # if no clash is found so return false
  return False


def crossover(chromosomes):
  # empty time table
  full_timetable = []

  # works on each section of chromosome
  for section_chromosomes in zip(*chromosomes):
    # randomly choose section and appply croosover
    selected_chromosome = random.choice(section_chromosomes)
    full_timetable.append(selected_chromosome)

  return full_timetable


def genetic_algorithm_crossover(population, num_generations):
  for generation in range(num_generations):
    # randomly select chromosomes from the population
    selected_chromosomes = random.sample(population, 2)
    # Perform crossover
    offspring = crossover(selected_chromosomes)

    # check fitness of that off spring
    if not Fitness(offspring):
        return offspring  # return if any clash is found

  return None  # Return None if no clash is found


def generate_timetable_with_genetic_algo(course_allocation):
  # Generate timetable entries
  timetable = generate_timetable(course_allocation)

  # Generate chromosomes for each section
  for section in set(course['Section'] for course in course_allocation):
    section_timetable = [entry for entry in timetable if entry.section == section]
    chromosomes = generate_chromosomes_for_section(section, section_timetable)
    population = []
    for i in range(5):  # Generate 5 chromosomes for each section
      population.append(chromosomes)
      print(f"Chromosomes {i+1} for Section {section}:")
      for chromosome in chromosomes:
        encoded_chromosome = encode_chromosome(chromosome)
        decoded_chromosome = decode_chromosome(encoded_chromosome)
        print(f"Original: {chromosome}")
        # print(f"Encoded: {encoded_chromosome}")
        # print(f"Decoded: {decoded_chromosome}")
        print()


# theory classes timing
def timeslot_to_theory_time(timeslot):
  theory_times = ["08:30-09:50", "10:05-11:25", "11:40-13:00", "13:15-14:35"]
  return theory_times[timeslot]


def timeslot_to_lab_time(timeslot):
  return "14:30-17:30"  # lab timing is fixed


In [87]:
from tabulate import tabulate

def print_timetable_for_sections_with_timings(timetable):
  sections = set(entry.section for entry in timetable)
  for section in sections:
      print(f"Timetable for Section {section}:")
      section_timetable = [entry for entry in timetable if entry.section == section]
      table_data = []
      for entry in section_timetable:
          days = ", ".join(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"][day] for day in entry.lecture_day)  # select the day from here.
          timing = ""
          if entry.is_lab:
              timing = timeslot_to_lab_time(entry.lecture_timeslot)
          else:
              timing = timeslot_to_theory_time(entry.lecture_timeslot)
          table_data.append([entry.course, entry.professor, days, timing, entry.room, entry.room_size])
      print(tabulate(table_data, headers=["Course", "Professor", "Days", "Time", "Room", "Room Size"]))
      print()

In [88]:
def print_timetable_for_sections_with_timings2(timetable):
  sections = set(entry.section for entry in timetable)
  for section in sections:
    print(f"Timetable for Section {section}:")
    section_timetable = [entry for entry in timetable if entry.section == section]
    for entry in section_timetable:
      days = entry.lecture_day
      timing = ""
      if entry.is_lab:
        timing = timeslot_to_lab_time(entry.lecture_timeslot)
        days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"][days[0]]
      else:
        timing = timeslot_to_theory_time(entry.lecture_timeslot)
        days = ", ".join(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"][day] for day in days)
      print(f"{entry} (Days: {days}, Time: {timing})")
      print()

In [89]:
def main():
  population = [generate_timetable(course_allocation) for _ in range(10)]  # generate initial population

  best_timetable = genetic_algorithm_crossover(population, num_generations=1000)
  if best_timetable:   # display the best time table
      print_timetable_for_sections_with_timings(best_timetable)
      print_timetable_for_sections_with_timings2(best_timetable)
  else:
      print("No clash free timetable can be found within the specified  generations :( ")

# main
if __name__ == "__main__":
  main()

Timetable for Section Section B:
Course                        Professor                    Days               Time         Room      Room Size
----------------------------  ---------------------------  -----------------  -----------  ------  -----------
Operating Systems (CS)        Dr. Ahmad Raza Shahid        Monday, Wednesday  10:05-11:25  C 221            60
Database Lab                  Prof. Usama Imtiaz           Monday             14:30-17:30  C 305            60
Artificial Intelligence (CS)  Prof. Muhammad Owais Idrees  Monday, Thursday   13:15-14:35  C 214           120

Timetable for Section Section C:
Course                        Professor                    Days              Time         Room      Room Size
----------------------------  ---------------------------  ----------------  -----------  ------  -----------
Operating Systems (CS)        Dr. Ahmad Raza Shahid        Thursday, Friday  13:15-14:35  C 246           120
Database Lab                  Prof. Usama Imtiaz